In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
lemmatizer = WordNetLemmatizer()

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


69 documents
9 classes ['charges_fees', 'foreclosure', 'goodbye', 'greetings', 'help', 'loan_application_status', 'thanks', 'top_Up_loan', 'want_personal_loan']
105 unique lemmatized words ["'s", ',', '.', 'a', 'about', 'account', 'adios', 'afternoon', 'again', 'all', 'am', 'application', 'appreciate', 'are', 'assistance', 'awesome', 'be', 'bill', 'but', 'bye', 'can', 'charge', 'charged', 'chatting', 'close', 'complain', 'could', 'day', 'did', 'do', 'doing', 'eligible', 'end', 'evening', 'fee', 'for', 'foreclose', 'get', 'going', 'good', 'goodbye', 'happy', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'know', 'late', 'later', 'like', 'loan', 'looking', 'may', 'me', 'more', 'morning', 'much', 'my', 'need', 'not', 'now', 'of', 'offered', 'ok', 'on', 'pay', 'payment', 'payoff', 'penalized', 'personal', 'processing', 'provide', 'regarding', 'sayonara', 'see', 'should', 'status', 'still', 'support', 'tell', 'thank', 'thanks', 'that', 'the', 'ther

In [8]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = [] # initializing bag of words
    pattern_words = doc[0]  # list of tokenized words for the pattern
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words] # lemmatize each word - create base word, in attempt to represent related words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)   # create our bag of words array with 1, if word match found in current pattern

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])


# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
#0 create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\ProgramData\Anaconda3\envs\BankBot\lib\site-packages\ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [9]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\ProgramData\Anaconda3\envs\BankBot\lib\site-packages\ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
14/14 [==============================] - 0s 2ms/step - loss: 2.1933 - accuracy: 0.1739
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 2.0849 - accuracy: 0.2464
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 1.9210 - accuracy: 0.3333
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 1.8010 - accuracy: 0.3478
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 1.6696 - accuracy: 0.3913
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 1.4372 - accuracy: 0.5652
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 1.2247 - accuracy: 0.6087
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 1.1809 - accuracy: 0.6087
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 0.9729 - accuracy: 0.7536
Epoch 10/200
14/14 [==============================] - 0s 2ms/step - loss: 0.8320 - accuracy: 0.7391
Epoch 11/

14/14 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 163/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 164/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 165/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0299 - accuracy: 0.9855
Epoch 166/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0266 - accuracy: 0.9855
Epoch 167/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0336 - accuracy: 0.9855
Epoch 168/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 169/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 170/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 1.0000
Epoch 171/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 1.0000
Epoc

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.5
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list



In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


In [26]:
import threading


In [27]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)



# Creating GUI with tkinter
from tkinter import *

base = Tk()
base.title("BankBot")
base.geometry("350x510")
base.resizable(width=True, height=True)

# Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)

#  Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


# Create Button to send message
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=2,
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command= send )

# Create the box to enter message
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial", )
# EntryBox.bind("<Return>", send)

# Place all components on the screen
scrollbar.place(x=335, y=5, height=445, width=10)
ChatLog.place(x=5, y=5, height=445, width=330)
ChatLog.config(state=NORMAL)
ChatLog.insert(END, "Bot: Welcome.!\nI am AssistBot,\nyour customer service assistant\nfor personal loan.\nHow may I help you.\n\n")
ChatLog.yview(END)
ChatLog.config(state=DISABLED)
EntryBox.place(x=130, y=455, height=50, width=210)
SendButton.place(x=5, y=455, height=50)


def enter_function(event):
    SendButton.invoke()


# going to bind main window with enter key
base.bind('<Return>', enter_function)

base.mainloop()
